# Indexes in Apache Pinot
## Table Creation 

In [589]:
!pip install pandas
import pandas
import json
import requests
import copy
import time

To compare tables with different indexes defined, we create multiple tables. The index configurations are applied to the tableIndexConfig-section of the *json_tableConfig* object. 

The tables are real-time tables and consume data from the same Kafka topic. Therefore, all of them will contain the same data records. To ensure, that the consumption of the tables has finished, we are monitoring when streaming data was ingested the last time to the table. Parameter minConsumingFreshnessTimeMs describes when the last ingestion took place. For the following use case we defined, that if there hasn't been any new data consumption in the last 15 seconds, the before generated data records of the Kafka topic have been consumed and there are enough data records to proceed.
(Remark: We are only checking the last updated time stamp for one specific table and assume, that there is no high time deviation for the different tables.)

In [622]:
table_list = []

json_tableConfig = {
  "tableName": "variable_tableName",
  "tableType": "REALTIME",
  "segmentsConfig": {
    "timeColumnName": "trip_start_time_millis",
    "timeType": "MILLISECONDS",
    "retentionTimeUnit": "DAYS",
    "retentionTimeValue": "60",
    "schemaName": "trips",
    "replication": "1",
    "replicasPerPartition": "1"
  },
  "tenants": {},
  "tableIndexConfig": {
    "loadMode": "MMAP",
    "streamConfigs": {
      "streamType": "kafka",
      "stream.kafka.consumer.type": "simple",
      "stream.kafka.topic.name": "trips",
      "stream.kafka.decoder.class.name": "org.apache.pinot.plugin.stream.kafka.KafkaJSONMessageDecoder",
      "stream.kafka.consumer.factory.class.name": "org.apache.pinot.plugin.stream.kafka20.KafkaConsumerFactory",
      "stream.kafka.zk.broker.url": "pinot-kafka-zookeeper:2181",
      "stream.kafka.broker.list": "pinot-kafka:9092",
      "realtime.segment.flush.threshold.time": "12h",
      "realtime.segment.flush.threshold.size": "20000",
      "stream.kafka.consumer.prop.auto.offset.reset": "smallest"
    }
  },
  "metadata": {
    "customConfigs": {}
  }
} 

def createTable(newTable_name, index_text, tableconfig_json):
    #Input: Name of new table, index description, table configuration in json structure
    response = requests.post('http://pinot-controller.pinot:9000/tables', json=tableconfig_json)
    print(response)
    print(response.text)
    table_list.append([newTable_name, index_text])


execution_start_time = int(round(time.time() * 1000))

# Create a new table with default index for each column (no configuration required)
newTable_defaultIndex = copy.deepcopy(json_tableConfig)
newTable_defaultIndex["tableName"] = "trips_default_index"
createTable(newTable_defaultIndex["tableName"], 'Default Index (Dictionary-encoded forward index with bit compression) for each column', newTable_defaultIndex)

# Create a new table with raw value forward index
newTable_rawForwardIndex = copy.deepcopy(json_tableConfig)
newTable_rawForwardIndex["tableName"] = "trips_rawForwardIndex"
newTable_rawForwardIndex["tableIndexConfig"]["noDictionaryColumns"] = ["start_location"]
createTable(newTable_rawForwardIndex["tableName"], 'Raw value forward index on start_location',newTable_rawForwardIndex)


# Create a new table with sorted forward index with run-length encoding
newTable_sortedForwardIndex = copy.deepcopy(json_tableConfig)
newTable_sortedForwardIndex["tableName"] = "trips_sortedForwardIndex"
newTable_sortedForwardIndex["tableIndexConfig"]["sortedColumn"] = ["start_location"]
createTable(newTable_sortedForwardIndex["tableName"],'Sorted forward index with run-length encoding on start location',newTable_sortedForwardIndex)


# Create a new table with bitmap inverted index
newTable_bitmapInvertedIndex2 = copy.deepcopy(json_tableConfig)
newTable_bitmapInvertedIndex2["tableName"] = "trips_bitmapInvertedIndex_startLocation"
newTable_bitmapInvertedIndex2["tableIndexConfig"]["invertedIndexColumns"] = ["start_location"]
createTable(newTable_bitmapInvertedIndex2["tableName"],'Bitmap inverted index on start_location',newTable_bitmapInvertedIndex2)


# Create a new table with sorted inverted index
newTable_sortedInvertedIndex2 = copy.deepcopy(json_tableConfig)
newTable_sortedInvertedIndex2["tableName"] = "trips_sortedInvertedIndex_startLocation"
newTable_sortedInvertedIndex2["tableIndexConfig"]["invertedIndexColumns"] = ["start_location"]
newTable_sortedInvertedIndex2["tableIndexConfig"]["sortedColumn"] = ["start_location"]
createTable(newTable_sortedInvertedIndex2["tableName"],'Sorted inverted index on start_location',newTable_sortedInvertedIndex2)

# Create a new table with star tree  index
newTable_starTree1 = copy.deepcopy(json_tableConfig)
newTable_starTree1["tableName"] = "trips_starTreeIndex"
newTable_starTree1["tableIndexConfig"]["starTreeIndexConfigs"] = [{
    "dimensionsSplitOrder": [
      "rider_is_premium",
      "start_location_state",
      "end_location"
    ],
    "functionColumnPairs": [
      "SUM__payment_amount",
    ],
    "maxLeafRecords": 1
  }]
createTable(newTable_starTree1["tableName"],'Star Tree 1',newTable_starTree1)

newTable_textIndex["tableIndexConfig"]["noDictionaryColumns"] = [
     "driver_name",
     "rider_name"
 ]
createTable(newTable_textIndex["tableName"],'Text Index',newTable_textIndex)

#Wait for tables to consume data for Kafka Topic
tableConsuming = True
while tableConsuming:
    response = requests.post('http://pinot-broker.pinot:8099/query/sql', json={
            "sql" : "SELECT * FROM trips_starTreeIndex"
        })
    response_json=response.json()
    if response_json['minConsumingFreshnessTimeMs']>0 and response_json['minConsumingFreshnessTimeMs']<(int(round(time.time() * 1000)) - 15000 ) or response_json['minConsumingFreshnessTimeMs']>9000000036854775807:
        tableConsuming = False;
        print("")
        print("--Consumption of generated data for trips_starTreeIndex finished--")


#Read value for start location and driver name for later queries
sql_get_values = "SELECT start_location, driver_name FROM " + newTable_defaultIndex["tableName"] 
response = requests.post('http://pinot-broker.pinot:8099/query/sql', json={
            "sql" : sql_get_values
    })
response_json=response.json()
startLocation = response_json['resultTable']['rows'][0][0]
#Only take the first name of the driver name
driverName = (response_json['resultTable']['rows'][0][1]).split()[0]
print(""+startLocation + " (start location) and " + driverName + " (driver name) will be used later as query variables.")

<Response [200]>
{"status":"Table trips_default_index_REALTIME succesfully added"}
<Response [200]>
{"status":"Table trips_rawForwardIndex_REALTIME succesfully added"}
<Response [200]>
{"status":"Table trips_sortedForwardIndex_REALTIME succesfully added"}
<Response [200]>
{"status":"Table trips_bitmapInvertedIndex_startLocation_REALTIME succesfully added"}
<Response [200]>
{"status":"Table trips_sortedInvertedIndex_startLocation_REALTIME succesfully added"}
<Response [200]>
{"status":"Table trips_starTreeIndex_REALTIME succesfully added"}
<Response [200]>
{"status":"Table trips_textIndex_REALTIME succesfully added"}

--Consumption of generated data for trips_starTreeIndex finished--
Stuttgart Stuttgart-Mitte (start location) and Alexis (driver name) will be used later as query variables.


### Table Size
Although the tables contain the same amount of records and also the same record values, the table size differs. This is because of the different indexes. E.g. a text index on one column is more space intensive compared to the raw forward index. The Star-Tree Index is allocating much space, as pre-aggregates defined calculations on metric columns.

In [624]:
for table in table_list:
    url_getsize = 'http://pinot-controller.pinot:9000/tables/' + table[0] + '_REALTIME/size?detailed=false'
    response = (requests.get(url_getsize)).json()
    print('tableName: ' + response['tableName'])
    print('Index Description: ' + table[1])
    v_size=response['reportedSizeInBytes']/1024/1024
    print('reportedSizeInMB: '+ str(v_size))
    print(" ")

tableName: trips_default_index_REALTIME
Index Description: Default Index (Dictionary-encoded forward index with bit compression) for each column
reportedSizeInMB: 22.090063095092773
 
tableName: trips_rawForwardIndex_REALTIME
Index Description: Raw value forward index on start_location
reportedSizeInMB: 24.462448120117188
 
tableName: trips_sortedForwardIndex_REALTIME
Index Description: Sorted forward index with run-length encoding on start location
reportedSizeInMB: 20.088777542114258
 
tableName: trips_bitmapInvertedIndex_startLocation_REALTIME
Index Description: Bitmap inverted index on start_location
reportedSizeInMB: 22.93588161468506
 
tableName: trips_sortedInvertedIndex_startLocation_REALTIME
Index Description: Sorted inverted index on start_location
reportedSizeInMB: 20.08920669555664
 
tableName: trips_starTreeIndex_REALTIME
Index Description: Star Tree 1
reportedSizeInMB: 39.382994651794434
 
tableName: trips_textIndex_REALTIME
Index Description: Text Index
reportedSizeInMB:

## Indexes in Apache Pinot

Pinot uses the MYSQL_ANSI dialect. Executing Joins or nested Subqueries is not supported. 
For accessing multiple tables in queries, the query engine Presto is recommended. In this report, we will focus on the functionality Pinot offers.
Pinot doesn't support data definition language. As already done, tables are created using the REST API.

Function *executeSQLStatement* takes a query string and an array containing table names and descriptions as input paramter. It executes the query on all tables which are defined in the table_list. If *specific_tables_array* is an empty string, the query will be executed on all tables which have been created for this chapter. The top two records of the result data set are displayed once to get an insight into the result data set. Additionaly, the function will create a dataframe listing execution metrics about the query execution for each table. Metrics of one query execution will only be appended to the dataframe if no exeption occurs. 

In [579]:
def executeSQLStatement(sql_statement_with_variable, specific_tables_array):
    pandas.set_option('display.max_colwidth', None)
    df_metrics = pandas.DataFrame(columns=['indextype','table', 'numDocsScanned',
       'numEntriesScannedInFilter', 'numEntriesScannedPostFilter',
       'totalDocs', 'timeUsedMs',
       'minConsumingFreshnessTimeMs',
       'exceptions'])
    b_resultRecordsNotShown = True;
    if not specific_tables_array:
        table_list_statement = table_list
    else:
        table_list_statement = specific_tables_array 
    for table in table_list_statement:
    
        sql_statement = sql_statement_with_variable.replace("XX_TABLE",table[0])
        sql_statement = sql_statement.replace("XX_STARTLOCATION","'"+startLocation+"'")
        sql_statement = sql_statement.replace("XX_DRIVERNAME","'"+driverName+"'") 
        response = requests.post('http://pinot-broker.pinot:8099/query/sql', json={
            "sql" : sql_statement
        })
        response_json=response.json()
        d = {'indextype': table[1], 'table': table[0],'numDocsScanned': [response_json['numDocsScanned']],'numDocsScanned': [response_json['numDocsScanned']],'numEntriesScannedInFilter': [response_json['numEntriesScannedInFilter']], 'numEntriesScannedPostFilter':[response_json['numEntriesScannedPostFilter']],'totalDocs':[response_json['totalDocs']],'timeUsedMs':[response_json['timeUsedMs']],'minConsumingFreshnessTimeMs':[response_json['minConsumingFreshnessTimeMs']],'exceptions':[response_json['exceptions']]}
        df_metrics_new = pandas.DataFrame(data=d)
        if not response_json['exceptions']:
             df_metrics = df_metrics.append(df_metrics_new,ignore_index=True)
       

        if b_resultRecordsNotShown:
            try:
                if not response_json['exceptions']:
                    columnNames = response_json['resultTable']['dataSchema']['columnNames']
                    rows = response_json['resultTable']['rows']

                    result_dataframe = pandas.DataFrame(columns=columnNames,data=rows)
                    print("Top two result records of: " + sql_statement )
                    display(result_dataframe.head(2))
                    b_resultRecordsNotShown = False
            except:
                pass

    print("Metrics of execution of: " + sql_statement_with_variable)
    display(df_metrics)


### Metrics
The most important metrics of a query execution for our analysis are:
- __timeUsedMs__: Total time between broker receiving the query request request and sending the response back to the client.
- __numDocScanned__: Number of documents/records scanned while query processing. (Includes records scanned in the filter phase as well as after applying the filter.)
- __numEntriesScannedInFilter__: It is an indicator of the latency contributed by the lookup phase. If this number is high, applying an index on the selection criteria might improve performance, especially if the column as high selectivity.
- __numEntriesScannedPostFilter__: High number is an indicator for low selectivity. Instead of regular indices, a star-tree index could help.

### Index Types
The before created tables contain the following Index Types of Apache Pinot:

- Forward Index
    - __Default Index: Dictionary-encoded forward index with bit compression__: 
    Apache Pinot will use this index by default for each column if no other index is configured in the table configuration metadata. An id is assigned to each unique value of the column, afterwards a dictionary is build from the id to the value. In the forward index, only the bit-compressed id of is persisted instead of the values. This compression improves space efficiency of the storage, if there are few unique values.
    
    <img src="images/RawValueForwardIndex.png" width="35%" height="35%">

    
    - __Raw Value Forward Index__: A raw value forward index is configured as a noDictionaryColumn in the table configuration file. Values instead of dictionary ids will be stored. Because of that, no dictionary lookup is required and due to the locality of values the performance of scanning large number of values is improved. 
    - __Sorted forward index with run-length encoding__: The sorted forward index is applied on top of the dictionary-encoding. For each dictionary id, a start and end document id is stored. Only one sorted column can be created for each Pinot table.
   
- Inverted Index: Inverted Indexes reduce the number of records which need be processed by identifying the ones which contain the search term. The inverted index is created by selecting all unique values of the column. For each value, a list of document ids which contain the value will be assigned. If we search e.g. for Hessen as a state, we can look up the inverted index for Hessen and identify the documents for which that value appears. 
    - __Bitmap inverted index__: A map from each value to a bitmap is maintained for the column which is enabled as bitmap inverted index. If a column is used frequently for filtering, an inverted index will improve the performance. (E.g. Thüringen -> Doc5, Doc1)
    - __Sorted inverted index__: A sorted index can benefit from data locality, but can only be applied to one column.
    
- __Star-Tree Index__: This index is built on multiple columns and pre-aggregates results, so that less values need to be processed. This can improve the query performance, on the other hand the pre-aggregation requires more storage (table size is about twice of the size as the other tables are).
- __Text Index__: Text Indexes in Pinot allow to do aribtrary search on STRING columns.

#### Default Index (Dictionary-encoded forward index with bit compression) vs Raw value forward index

In [580]:
executeSQLStatement("select count, driver_name, driver_rating, end_location, end_location_state from XX_TABLE WHERE start_location=XX_STARTLOCATION",[['trips_default_index', 'Default Index (Dictionary-encoded forward index with bit compression) for each column'],['trips_rawForwardIndex', 'Raw value forward index on start_location']])

Top two result records of: select count, driver_name, driver_rating, end_location, end_location_state from trips_default_index WHERE start_location='Rietz-Neuendorf'


,count,driver_name,driver_rating,end_location,end_location_state
0,1,Gloria Hayden,1,Rückersdorf,Bayern
1,1,Vivian Bryant,4,Hanau,Hessen


Metrics of execution of: select count, driver_name, driver_rating, end_location, end_location_state from XX_TABLE WHERE start_location=XX_STARTLOCATION


,indextype,table,numDocsScanned,numEntriesScannedInFilter,numEntriesScannedPostFilter,totalDocs,timeUsedMs,minConsumingFreshnessTimeMs,exceptions
0,Default Index (Dictionary-encoded forward index with bit compression) for each column,trips_default_index,10,11441,50,308637,6,1618057229495,[]
1,Raw value forward index on start_location,trips_rawForwardIndex,10,7766,50,308637,11,1618057229492,[]


The query execution on table __trips_rawForwardIndex (1)__ takes more time. This is because more entries need to be scanned in the filter.  For table __trips_default_index__, less records are scanned. The main difference between the two index types is, that the index on column start_location of __trips_default_index (0)__ creates dictionaries. The dictionary provides compression when values of the columns occurr repeatedly. 
A dictionary index doesn't provide that advantage anymore, if a column consists of lots of unique values.

#### Default Index (Dictionary-encoded forward index with bit compression) vs Sorted forward index with run-length encoding

In [581]:
executeSQLStatement("select * from XX_TABLE WHERE start_location=XX_STARTLOCATION", [['trips_default_index', 'Default Index (Dictionary-encoded forward index with bit compression) for each column'],['trips_sortedForwardIndex', 'Sorted forward index with run-length encoding on start location']])

Top two result records of: select * from trips_default_index WHERE start_location='Rietz-Neuendorf'


,count,driver_name,driver_rating,end_location,end_location_state,end_zip_code,license_plate,payment_amount,payment_tip_amount,request_time_millis,rider_is_premium,rider_name,rider_rating,start_location,start_location_state,start_zip_code,trip_end_time_millis,trip_start_time_millis,trip_wait_time_millis
0,1,Gloria Hayden,1,Rückersdorf,Bayern,90607,EC-AK-00,33.35,13.0,1618269534009,0,Darryl Prieto,4,Rietz-Neuendorf,Brandenburg,15848,1618271937469,1618269929777,395768
1,1,Vivian Bryant,4,Hanau,Hessen,63450,ZQ-NE-67,345.80,16.0,1618271419019,0,Lawanda Ramsey,0,Rietz-Neuendorf,Brandenburg,15848,1618296034464,1618271605893,186874


Metrics of execution of: select * from XX_TABLE WHERE start_location=XX_STARTLOCATION


,indextype,table,numDocsScanned,numEntriesScannedInFilter,numEntriesScannedPostFilter,totalDocs,timeUsedMs,minConsumingFreshnessTimeMs,exceptions
0,Default Index (Dictionary-encoded forward index with bit compression) for each column,trips_default_index,10,11441,190,308637,12,1618057229495,[]
1,Sorted forward index with run-length encoding on start location,trips_sortedForwardIndex,10,0,190,308637,14,1618057229510,[]


Comparison of __trips_default_index (0)__ and __trips_sortedInvertedIndex_startLocation (1)__:

XXXXXX The sorted forward index on column start_location of table __trips_sortedInvertedIndex_startLocation (1)__ has the advantage of data locality. Because of this, numEntriesScannedInFilter is less than for the column with default index.
Query execution time can be faster when using the sorted forward index on column start_location.

#### Default Index (Dictionary-encoded forward index with bit compression) vs Inverted index (Bitmap + Sorted)

In [582]:
executeSQLStatement("select driver_name, rider_name from XX_TABLE WHERE start_location=XX_STARTLOCATION", [['trips_default_index', 'Default Index (Dictionary-encoded forward index with bit compression) for each column'],['trips_bitmapInvertedIndex_startLocation', 'Bitmap inverted index on start_location'],['trips_sortedInvertedIndex_startLocation','Sorted inverted index on start_location']])

Top two result records of: select driver_name, rider_name from trips_default_index WHERE start_location='Rietz-Neuendorf'


,driver_name,rider_name
0,Gloria Hayden,Darryl Prieto
1,Vivian Bryant,Lawanda Ramsey


Metrics of execution of: select driver_name, rider_name from XX_TABLE WHERE start_location=XX_STARTLOCATION


,indextype,table,numDocsScanned,numEntriesScannedInFilter,numEntriesScannedPostFilter,totalDocs,timeUsedMs,minConsumingFreshnessTimeMs,exceptions
0,Default Index (Dictionary-encoded forward index with bit compression) for each column,trips_default_index,10,11441,20,308637,5,1618057229495,[]
1,Bitmap inverted index on start_location,trips_bitmapInvertedIndex_startLocation,10,0,20,308637,8,1618057229499,[]
2,Sorted inverted index on start_location,trips_sortedInvertedIndex_startLocation,10,0,20,308637,10,1618057229492,[]


Comparison of __trips_default_index (0)__ and __trips_bitmapInvertedIndex_startLocation	 (4)__:

Bitmap Inverted Index ( __trips_bitmapInvertedIndex_startLocation (5)__)
An inverted index can improve the query performance. In this case, no entries have to be scanned in filter and the query runtime is faster compared to using the dictionary encoded index (0) or a raw value forward index (1).

By using the sorted inverted index, the performance can benefit from data locality. 
When grouping by start_location, query execution on __trips_bitmapInvertedIndex_startLocation__	 and __trips_sortedInvertedIndex_startLocation__	 can perform faster than than trips_default_index.

#### Text Search Support

A query searching for all driver name, which contain a specfic first name, will only run successful on table trips_textIndex, as it has an index defined on column driver_name. The same query execution on other tables will fail, the metrics table only displays the executions without an exception.

In [583]:
executeSQLStatement("select * from XX_TABLE WHERE TEXT_MATCH ('driver_name',XX_DRIVERNAME) LIMIT 50000", [['trips_textIndex','Text Index']])

Top two result records of: select * from trips_textIndex WHERE TEXT_MATCH ('driver_name','Gloria') LIMIT 50000


,count,driver_name,driver_rating,end_location,end_location_state,end_zip_code,license_plate,payment_amount,payment_tip_amount,request_time_millis,rider_is_premium,rider_name,rider_rating,start_location,start_location_state,start_zip_code,trip_end_time_millis,trip_start_time_millis,trip_wait_time_millis
0,1,Gloria Driver,3,Ehningen,Baden-Württemberg,71139,US-TV-61,1224.45,48.0,1618110080018,0,William Orgill,1,Auhagen Düdinghausen,Niedersachsen,31553,1618167891657,1618113471657,3391639
1,1,Gloria Hayden,3,Merzig,Saarland,66663,TL-SC-32,1624.86,10.0,1618110366016,1,Valerie Kuznicki,2,Tiefenthal,Rheinland-Pfalz,67311,1618172832462,1618110477745,111729


Metrics of execution of: select * from XX_TABLE WHERE TEXT_MATCH ('driver_name',XX_DRIVERNAME) LIMIT 50000


,indextype,table,numDocsScanned,numEntriesScannedInFilter,numEntriesScannedPostFilter,totalDocs,timeUsedMs,minConsumingFreshnessTimeMs,exceptions
0,Text Index,trips_textIndex,621,0,11799,308637,132,1618057229497,[]


#### Star-Tree Index
The Start-Tree index is utilizes pre-aggregation of results and is built on multiple columns. This index can improve the performance for specific queries, because the number of values to be processed is reduced by the pre-aggregation. Although usage of a Star-Tree index has the advantage of decreased query runtime, the table size is increased.
For table trips_starTreeIndex, a Star-Tree Index is built on the dimensions rider_is_premium, start_location_state and end_location. The sum of payment_amount and is be pre-aggregated and materialized based on this dimensions.

In [584]:
executeSQLStatement("SELECT SUM(payment_amount) FROM XX_TABLE",[["trips_default_index","Default Index (Dictionary-encoded forward index with bit compression) for each column"],["trips_starTreeIndex", "Start Tree 1"]] )

Top two result records of: SELECT SUM(payment_amount) FROM trips_default_index


,sum(payment_amount)
0,2.171565e+08


Metrics of execution of: SELECT SUM(payment_amount) FROM XX_TABLE


,indextype,table,numDocsScanned,numEntriesScannedInFilter,numEntriesScannedPostFilter,totalDocs,timeUsedMs,minConsumingFreshnessTimeMs,exceptions
0,Default Index (Dictionary-encoded forward index with bit compression) for each column,trips_default_index,308637,0,308637,308637,17,1618057229495,[]
1,Start Tree 1,trips_starTreeIndex,8652,0,8652,308637,7,1618057229493,[]


When selecting the Star-Node without grouping by any dimension, Pinot doesn't need to access all documents. Instead, only the Star-Node of each segment is required. The reason, why numDocsScanned is unequal the number of segments is, that one segment isn't completed yet. Pinot accesses each segment and each records of the segment which still has the status IN_PROGRESS.

In [585]:
print("The table with the Star Tree Index consists of : " + str(len(requests.get('http://pinot-controller.pinot:9000/segments/trips_starTreeIndex').json()[0]['REALTIME'])) + " segments")

The table with the Star Tree Index consists of : 16 segments


Filter on the dimension rider_is_premium, which build the first node of the Star Tree Index, halves the number of numDocsScanned. This is because rider_is_premiumn is assigned randomly in our data generation, so there is a fifty percent chance that a rider is not premium and less documents of the not finished segment need to be scanned.

In [586]:
executeSQLStatement("SELECT SUM(payment_amount) FROM XX_TABLE WHERE rider_is_premium = 0",[["trips_default_index","Default Index (Dictionary-encoded forward index with bit compression) for each column"],["trips_starTreeIndex", "Start Tree 1"]])

Top two result records of: SELECT SUM(payment_amount) FROM trips_default_index WHERE rider_is_premium = 0


,sum(payment_amount)
0,1.085799e+08


Metrics of execution of: SELECT SUM(payment_amount) FROM XX_TABLE WHERE rider_is_premium = 0


,indextype,table,numDocsScanned,numEntriesScannedInFilter,numEntriesScannedPostFilter,totalDocs,timeUsedMs,minConsumingFreshnessTimeMs,exceptions
0,Default Index (Dictionary-encoded forward index with bit compression) for each column,trips_default_index,154446,308637,154446,308637,20,1618057229495,[]
1,Start Tree 1,trips_starTreeIndex,4375,8637,4375,308637,11,1618057229493,[]


##### Trace Details 
The trace details of the query execution display how much time was spend for which operator execution. We extract the operator details in the following. The query executed on trips_default_index requires a lot of Aggregation Operators, as no data is preaggregated like it is the case for the table trips_starTreeIndex.

In [587]:
def query_sql(query):
    print("query: " + query)
    return requests.get('http://pinot-broker.pinot:8099/query/sql', params={
        "sql" : query,
        "trace": "true"
    }).json()
def getTraceDetails(sql):
    response = query_sql(sql)
    test = (response['traceInfo']['pinot-server-0.pinot-server-headless.pinot.svc.cluster.local'])[7:-3]
    string_tracedetails = test.split(',')
    # display only trace details with time > 0
    tracedetails_time = []
    for x in string_tracedetails:
        if "Time" in x:
            if ":0" in x:
                pass
            else:
                tracedetails_time.append(x)
    print(tracedetails_time)

print("\nGet Trace Details for trips_starTreeIndex\n")
getTraceDetails("SELECT SUM(payment_amount) FROM trips_starTreeIndex WHERE rider_is_premium = 0")
print("\nGet Trace Details for trips_default_index\n")
getTraceDetails("SELECT SUM(payment_amount) FROM trips_default_index WHERE rider_is_premium = 0")


Get Trace Details for trips_starTreeIndex

query: SELECT SUM(payment_amount) FROM trips_starTreeIndex WHERE rider_is_premium = 0
['{"AggregationOnlyCombineOperator Time":2}', '{"InstanceResponseOperator Time":2}]}', '{"AggregationOperator Time":1}', '{"DocIdSetOperator Time":1}', '{"ProjectionOperator Time":1}', '{"TransformOperator Time":1}', '{"AggregationOperator Time":1}']

Get Trace Details for trips_default_index

query: SELECT SUM(payment_amount) FROM trips_default_index WHERE rider_is_premium = 0
['{"AggregationOnlyCombineOperator Time":14}', '{"InstanceResponseOperator Time":14}]}', '{"DocIdSetOperator Time":1}', '{"ProjectionOperator Time":1}', '{"TransformOperator Time":1}', '{"AggregationOperator Time":1}', '{"AggregationOperator Time":1}', '{"DocIdSetOperator Time":1}', '{"ProjectionOperator Time":1}', '{"TransformOperator Time":1}', '{"AggregationOperator Time":1}', '{"DocIdSetOperator Time":1}', '{"ProjectionOperator Time":1}', '{"TransformOperator Time":1}', '{"Aggrega

### Indexes - Comparison with other database technologies
Two main types of indexes in Apache Pinot are Forward Indexes and Inverted Indexes.
Forward Indexes are most often used in traditional databases.
Search Engines most often rely on the inverted index, one example is Elasticsearch.

### Maintenance - Table Deletion

In [621]:
## Delete Tables
for table in table_list:
    string = "http://pinot-controller.pinot:9000/tables/" + table[0]
    response = requests.delete(string)
    print(response.json())
    table_list = []

{'status': 'Tables: [trips_default_index_REALTIME] deleted'}
{'status': 'Tables: [trips_rawForwardIndex_REALTIME] deleted'}
{'status': 'Tables: [trips_sortedForwardIndex_REALTIME] deleted'}
{'status': 'Tables: [trips_bitmapInvertedIndex_startLocation_REALTIME] deleted'}
{'status': 'Tables: [trips_sortedInvertedIndex_startLocation_REALTIME] deleted'}
{'status': 'Tables: [trips_starTreeIndex_REALTIME] deleted'}
{'status': 'Tables: [trips_textIndex_REALTIME] deleted'}
